In [1]:
import numpy as np
import math

In [18]:
class QuasiMonteCarloOption:
    def __init__(self, S_0, K, r, sigma, T, option_type, dim):
        """
        Инициализация основных параметров опциона.
        S: Цена базового актива
        K: Цена страйка
        r: Безрисковая ставка
        sigma: Волатильность
        T: Время до истечения опциона
        option_type: "call" для колл-опциона, "put" для пут-опциона.
        B: Уровень барьера (если требуется для барьерных опционов)
        t: Текущее время
        """
        self.S_0 = S_0
        self.K = K
        self.r = r
        self.sigma = sigma
        self.T = T
        self.option_type = option_type
        self.dim = dim
        
    def box_muller(self, U1, U2):
        # преобразование U1 и U2 в массивы NumPy, если они не являются таковыми
        U1 = np.asarray(U1)
        U2 = np.asarray(U2)
        
        R = np.sqrt(-2 * np.log(U1))
        Theta = 2 * np.pi * U2
        
        arr_Z = R * np.cos(Theta)  # генерация нормально распределенных значений
        return arr_Z
        
    def european_option(self, sequence, t):
        '''
        Вычисление премии европейского опциона (колл или пут).
        option_type: "call" для колл-опциона, "put" для пут-опциона.
        '''
        tau = self.T - t
        delta_t = tau / (self.dim - 1)
        n = int(len(sequence) / 2)
    
        U1 = sequence[:n]
        U2 = sequence[n:2 * n]
        arr_Z = self.box_muller(U1, U2)
    
        mean_C = 0
        for i in range(n):
            S_i = self.S_0
            for j in range(self.dim - 1):
                S_i = S_i * np.exp((self.r - 0.5 * self.sigma**2) * delta_t + self.sigma * np.sqrt(delta_t) * arr_Z[i][j])
            if self.option_type == "call":
                C_i = np.exp(-self.r * T) * max(S_i - self.K, 0)
            elif self.option_type == "put":
                C_i = np.exp(-self.r * T) * max(self.K - S_i, 0)
            mean_C += C_i
    
        return mean_C / n



    def asian_option(self, sequence, t):
        '''
        Вычисление премии азиатского опциона (колл или пут).
        option_type: "call" для колл-опциона, "put" для пут-опциона.
        '''
        tau = self.T - t
        delta_t = tau / (self.dim - 1)
        n = int(len(sequence) / 2)
    
        U1 = sequence[:n]
        U2 = sequence[n:2 * n]
        arr_Z = self.box_muller(U1, U2)
    
        arr_C = []
        for i in range(n):
            arr_S_i = []
            for j in range(self.dim - 1):
                if j == 0:
                    S_i = self.S_0 * np.exp((self.r - 0.5 * self.sigma**2) * delta_t + self.sigma * np.sqrt(delta_t) * arr_Z[i][j])
                else:
                    S_i = arr_S_i[j - 1] * np.exp((self.r - 0.5 * self.sigma**2) * delta_t + self.sigma * np.sqrt(delta_t) * arr_Z[i][j])
                arr_S_i.append(S_i)
    
            S_mean = np.mean(arr_S_i)
            if self.option_type == "call":
                C_i = np.exp(-self.r * T) * max(S_mean - self.K, 0)
            elif self.option_type == "put":
                C_i = np.exp(-self.r * T) * max(self.K - S_mean, 0)
            arr_C.append(C_i)
    
        return np.mean(arr_C)


    
    def hitting_probability(self, S, tau, B):
    
        k = (math.log(S / self.S_0) - (self.r - 0.5 * self.sigma**2) * tau) / self.sigma
        m = (math.log(B / self.S_0) - (self.r - 0.5 * self.sigma**2) * tau) / self.sigma
        prob = 1 - np.exp(2*m*(k-m)/tau)
        return prob

        
    def up_and_out_option(self, sequence, B, t): # проверить hitting_probability
        '''
        Вычисление премии барьерного up-and-out опциона (колл или пут).
        option_type: "call" для колл-опциона, "put" для пут-опциона.
        '''
        tau = self.T - t
        delta_t = tau / (self.dim - 1)
        n = int(len(sequence) / 2)
    
        U1 = sequence[:n]
        U2 = sequence[n:2 * n]
        arr_Z = self.box_muller(U1, U2)
    
        total_payoff = 0
        for i in range(n):
            breached = False
            S_i = self.S_0
    
            for j in range(self.dim - 1):
                arr_S_i = [S_i]
                
                S_i = S_i * np.exp((self.r - 0.5 * self.sigma**2) * delta_t + self.sigma * np.sqrt(delta_t) * arr_Z[i][j])
                
                arr_S_i.append(S_i)
                
                if S_i > B:
                    breached = True
                    break
    
            if not breached:
                if self.option_type == "call":
                    S_max = np.array(arr_S_i).max()
                    prob = self.hitting_probability(S_max, delta_t, B)
                    C_i = np.exp(-self.r * tau) * max((S_i - self.K)*prob, 0)
                elif self.option_type == "put":
                    S_max = np.array(arr_S_i).max()
                    prob = self.hitting_probability(S_max, delta_t, B)
                    C_i = np.exp(-self.r * tau) * max((self.K - S_i)*prob, 0)
            else:
                C_i = 0
    
            total_payoff += C_i
    
        return total_payoff / n

        
    def lookback_option(self, sequence, y, t):
        '''
        Вычисление средней стоимости lookback опциона (колл или пут).
        option_type: "call" для колл-опциона, "put" для пут-опциона.
        
        x = S
        y = S_max
        
        '''
        x = self.S_0
        y = S_max
        
        tau = self.T - t
        t_values = np.linspace(0, tau, self.dim + 1)
        payoff_sum = 0

      
        n = int(len(sequence) / 2)
        U1 = sequence[:n]
        U2 = sequence[n:2 * n]
        arr_Z = self.box_muller(U1, U2)
    
        for i in range(n):
            S_i = x
            max_S_i = y
            min_S_i = y
    
            for j in range(self.dim):
                delta_t = t_values[j + 1] - t_values[j]
                S_i = S_i * np.exp((self.r - 0.5 * self.sigma**2) * delta_t + self.sigma * np.sqrt(delta_t) * arr_Z[i][j])
                max_S_i = max(max_S_i, S_i)
                min_S_i = min(min_S_i, S_i)
    
            if self.option_type == "call":
                C_i = np.exp(-self.r * tau) * (max_S_i - S_i)
            elif self.option_type == "put":
                C_i = np.exp(-self.r * tau) * (S_i - min_S_i)
            payoff_sum += C_i
    
        return payoff_sum / n


In [19]:
# from scipy.stats import qmc               


In [20]:
# dim = 5
# m = 7
# seq = qmc.Sobol(d=dim, scramble=False).random_base2(m+1)[1:2**m+1]

In [21]:
# dim = 5
# K=95
# S_0=100
# sigma =0.2
# r = 0.21
# option_type = 'call'
# # option_type = 'put'
# B=1000000000
# t=0
# T=1
# S_max = 110
# qmc0 =  QuasiMonteCarloOption(S_0, K, r, sigma, T, option_type, dim)
# qmc0.european_option(seq, 0.5)

14.124496426667147

In [23]:
# time_steps = np.linspace(0, T, 11, endpoint=True)  
# # qmc0 =  QuasiMonteCarloOption(S_0, K, r, sigma, T, option_type, dim)

# result_european_option, result_up_and_out_option, result_lookback_option, result_asian_option = [], [], [], []
# for t_step in time_steps:
    
#     print(f'{t_step}')

#     result_european_option.append(qmc0.european_option(seq, t_step))

0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9
1.0


In [24]:
# result_european_option

[24.247034750997116,
 22.157919423512688,
 20.103192841805203,
 18.080230782345918,
 16.089786531377396,
 14.124496426667147,
 12.174985396219771,
 10.230153326763293,
 8.268760837029872,
 6.237894824106584,
 4.052921229850937]